In [261]:
import pandas as pd
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [262]:
data = pd.read_csv('/home/pomoreq/eCommerceCustomerBehaviorAnalysis/data/data.csv', encoding='unicode_escape')
print(data)

       InvoiceNo StockCode                          Description  Quantity      InvoiceDate  UnitPrice  CustomerID         Country
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   12/1/2010 8:26       2.55     17850.0  United Kingdom
1         536365     71053                  WHITE METAL LANTERN         6   12/1/2010 8:26       3.39     17850.0  United Kingdom
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   12/1/2010 8:26       2.75     17850.0  United Kingdom
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   12/1/2010 8:26       3.39     17850.0  United Kingdom
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   12/1/2010 8:26       3.39     17850.0  United Kingdom
...          ...       ...                                  ...       ...              ...        ...         ...             ...
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12  12/9/2011 12:50

1.Preprocessing kolumn oraz stukturalna EDA
1A. kolumna invoiceNo

Sprawdzam czy są nany.
0 nanow

In [263]:
data['InvoiceNo'] = data['InvoiceNo'].astype('string')
isNansInvoiceNo = data['InvoiceNo'].isna().sum()

typ danych poprawny?
Nie, zmieniam z typu danych obiekt na string ponieważ indetyfikatory zawieraja znaki nienumeryczne

In [264]:
data['InvoiceNo'] = data['InvoiceNo'].astype('string')
data.dtypes

InvoiceNo      string[python]
StockCode              object
Description            object
Quantity                int64
InvoiceDate            object
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

Czy są powtorzenia indeytfikatora?
Tak są np faktura 573835 wystepuje 1114 razy. oznacza to ze jedna faktura to wiele pozycji

In [265]:
valueCountsInvoiceNo = data['InvoiceNo'].value_counts().sort_values(ascending=False)

 Czy długości InvoiceNo są spójne?
 Tak dla zamowienia jest dlugosc 6 a dla zwrotu 7

In [266]:
lengthOfValuesInInvoiceNo = data['InvoiceNo'].apply(len).value_counts()
lengthOfValuesInInvoiceNo

InvoiceNo
6    532618
7      9291
Name: count, dtype: int64

 Czy są dziwne znaki/spacje?
 Nie ma, wnioskuje po dlugosci wartosci

Dodaje kolumne isRefund która oznacza czy zamowienie jest zwrotem

In [267]:
data['isRefund'] = data['InvoiceNo'].str[0] == 'C'


Czy invoiceNo jest alfaNumeryczny
Tak invoice no jest alfa numeryczny

In [268]:
isInvoiceNoAlnum = data['InvoiceNo'].apply(lambda s: s.isalnum()).all()
isInvoiceNoAlnum

np.True_

Ile pozycji zawiera przeciętny InvoiceNo?
(średnia liczba rekordów przypadających na jeden numer faktury)
srednia liczba rekordow to18,3

In [269]:
meanPositionsForInvoiceNo = data.groupby('InvoiceNo')['StockCode'].count().mean()
meanPositionsForInvoiceNo

np.float64(20.923127413127414)

Czy są InvoiceNo, które mają tylko jedną pozycję?
Czy są faktury jednoitemowe?
tak istnieja takie faktury

In [270]:
isOneItemInvoiceNo = (data.groupby('InvoiceNo')['StockCode'].count() == 1).any()
isOneItemInvoiceNo

np.True_

Czy InvoiceNo są uporządkowane rosnąco względem daty?
tak bo pierwszy wiersz pokrywa sie z invoiceDateMin a ostatni z max

In [271]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,False
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,False
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,False
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,False
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,False
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,False


Czy są duże grupy duplikatów InvoiceNo z różnymi CustomerID?
czy są przypadki, że ten sam InvoiceNo ma przypisanych różnych klientów?
jeden invoice no ma jesten customerid

In [308]:
isInvoiceNoWithMultipleClients = data.groupby('InvoiceNo')['CustomerID'].nunique()
isInvoiceNoWithMultipleClients

InvoiceNo
536365     1
536366     1
536367     1
536368     1
536369     1
          ..
C581484    1
C581490    1
C581499    1
C581568    1
C581569    1
Name: CustomerID, Length: 22190, dtype: int64

Duplikaty w danych?
tak wystepuja duplikaty czyli invoiceNo z tym samym stocCodem np. invoice no 536409 ma dwa razy 90199C. do dalszej analizy czy jeden stockcode moze miec wiele opisow tzn np rozniace sie koolory

In [ ]:
isDuplicatedInvoiceNoAndStockCode = data.duplicated(subset=['InvoiceNo','StockCode']) == True
onlyTrues = isDuplicatedInvoiceNoAndStockCode[isDuplicatedInvoiceNoAndStockCode == True]
onlyTruesIndexes = onlyTrues.index
duplicates = data.loc[onlyTruesIndexes,['InvoiceNo','StockCode']]
duplicates
onlyTrues

125       True
498       True
502       True
517       True
525       True
          ... 
541692    True
541697    True
541698    True
541699    True
541701    True
Length: 10148, dtype: bool

1B. Kolumna StockCode

Czy są nany?
0 nanów

In [273]:
isNansStockCode = data['StockCode'].isna().sum()
isNansStockCode

np.int64(0)

Czy jest prawidlowy typ danych?
Nie, jest object a mozna zmienic na String. poniewaz StockCode zawiera litery oraz cyfry

In [274]:
data['StockCode'] = data['StockCode'].astype('string')

Czy są podejrzane kody (np. bardzo krótkie, bardzo długie)
Tak jest przestrzal dlugosci kodow od 1 do 12.

In [275]:
lenOfValuesStockCode = data['StockCode'].apply(len).value_counts()
lenOfValuesStockCode


StockCode
5     487036
6      51488
4       1276
1        715
3        710
7        390
2        144
12        71
9         48
8         31
Name: count, dtype: int64

Czy występują znaki nietypowe, spacje, znaki specjalne?
tak,wystepuja znaki specjalne, zostawiam je ze wzgledu na to ze to sa np indetyfikatory giftow. kody zwieraja cyfry,litery i znaki specjalne

In [276]:
isAlnumStockCode = data['StockCode'].apply(lambda s: s.isalnum()).all()
nonAlnum = data[~data['StockCode'].apply(str.isalnum)]
nonAlnum

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund
4406,536779,BANK CHARGES,Bank Charges,1,12/2/2010 15:08,15.00,15823.0,United Kingdom,False
14435,C537572,BANK CHARGES,Bank Charges,-1,12/7/2010 12:00,95.38,NaN,United Kingdom,True
28992,C538680,BANK CHARGES,Bank Charges,-1,12/13/2010 17:10,966.92,NaN,United Kingdom,True
38248,539492,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,1,12/20/2010 10:14,34.04,NaN,United Kingdom,False
42057,539958,gift_0001_50,Dotcomgiftshop Gift Voucher £50.00,1,12/23/2010 13:26,42.55,NaN,United Kingdom,False
...,...,...,...,...,...,...,...,...,...
481152,C577342,BANK CHARGES,Bank Charges,-1,11/18/2011 15:11,490.06,NaN,United Kingdom,True
481153,C577343,BANK CHARGES,Bank Charges,-1,11/18/2011 15:13,27.21,NaN,United Kingdom,True
506357,579137,BANK CHARGES,Bank Charges,1,11/28/2011 12:51,15.00,14704.0,United Kingdom,False
534008,581127,BANK CHARGES,Bank Charges,1,12/7/2011 12:45,15.00,16271.0,United Kingdom,False


Czy są kody zawierające tylko litery / liczby / znaki specjalne?
Są kody zwierajace tylko liczby, sa kody zwierajace tylko tez literyi, nie ma stock code zlozonych tylkoze znakow specjalnych
Czy są StockCode o nietypowych nazwach?
Tak są, typu post,dot

In [322]:
isOnlyLetters = data['StockCode'].apply(lambda s: s.isalpha()).any()
isOnlyLetters

np.True_

In [323]:
isOnlyLettersCheck = data['StockCode'].apply(lambda s: s.isalpha())
isOnlyLettersCheckTrueIndices = isOnlyLettersCheck[isOnlyLettersCheck == True].index
isOnlyLettersCheckRows = data.loc[isOnlyLettersCheckTrueIndices,'StockCode']
isOnlyLettersCheckRows

45        POST
141          D
386       POST
1123      POST
2239         M
          ... 
541216    POST
541541       M
541730    POST
541767    POST
541768    POST
Name: StockCode, Length: 1774, dtype: string

In [341]:
isOnlySpecial = data['StockCode'].apply(lambda s: all((not c.isalnum() for c in str(s))))
isOnlySpecial

0         False
1         False
2         False
3         False
4         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Name: StockCode, Length: 406829, dtype: bool

In [342]:
isOnlySpecial = data['StockCode'].apply(lambda s:  all((not c.isalnum() for c in str(s))))
isOnlySpecialTrueIndices = isOnlySpecial[isOnlySpecial == True].index
isOnlySpecialCheckRows = data.loc[isOnlySpecialTrueIndices,'StockCode']
isOnlySpecialCheckRows

Series([], Name: StockCode, dtype: string)

sprawdzam ilość unikalnych kodów
4070 unikalnych kodów

In [277]:
nUniqueStockCode = data['StockCode'].nunique()
nUniqueStockCode

4070

Czy niektóre kody występują wyjątkowo często?
są co wyczestupa czesto, ale nie wyjatkowo czesto
Czy są StockCode, które występują tylko raz w całym zbiorze?Czy sa podejrzane
tak są, ne sa podejrzane.

In [310]:
stockCodeValueCounts = data['StockCode'].value_counts().sort_values(ascending=False)
stockCodeValueCounts

StockCode
85123A    2077
22423     1905
85099B    1662
84879     1418
47566     1416
          ... 
90027A       1
90027D       1
90214Z       1
84206B       1
23449        1
Name: count, Length: 3684, dtype: Int64

Czy jeden StockCode może mieć więcej niż jeden Description?
Tak kod moze miec wiecej desciprtion, opisy roznia sie kolorem lub szykiem zdania opisu

In [347]:
stockCodeAndDescription = data.groupby('StockCode')['Description'].nunique().sort_values(ascending=False)
stockCodeAndDescription

StockCode
23236    4
23196    4
23231    3
22937    3
23413    3
        ..
22358    1
22359    1
22360    1
22361    1
22346    1
Name: Description, Length: 3684, dtype: int64

In [353]:
moreThanThreeDescCodes = stockCodeAndDescription[stockCodeAndDescription > 2].index
filteredThanThreeDescCodes = data[data['StockCode'].isin(moreThanThreeDescCodes)]
filteredThanThreeDescCodes
StockCode23535 = filteredThanThreeDescCodes[filteredThanThreeDescCodes['StockCode'] == '23535']
StockCode23535

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund,InvoiceYear,InvoiceMonth,InvoiceWeekday,IsWeekend,IsUK
371806,569227,23535,BICYCLE SAFTEY WALL ART,4,2011-10-02 14:37:00,5.95,14051,United Kingdom,False,2011,10,Sunday,False,True
373008,569255,23535,BICYCLE SAFTEY WALL ART,72,2011-10-03 11:01:00,3.88,18102,United Kingdom,False,2011,10,Monday,False,True
373020,569257,23535,BICYCLE SAFTEY WALL ART,36,2011-10-03 11:24:00,3.88,18102,United Kingdom,False,2011,10,Monday,False,True
373102,569261,23535,BICYCLE SAFTEY WALL ART,2,2011-10-03 11:34:00,5.95,17321,United Kingdom,False,2011,10,Monday,False,True
373582,569333,23535,BICYCLE SAFTEY WALL ART,4,2011-10-03 13:49:00,5.95,13431,United Kingdom,False,2011,10,Monday,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537879,581375,23535,WALL ART BICYCLE SAFETY,8,2011-12-08 12:36:00,5.95,16210,United Kingdom,False,2011,12,Thursday,True,True
538109,581399,23535,WALL ART BICYCLE SAFETY,4,2011-12-08 13:27:00,5.95,17528,United Kingdom,False,2011,12,Thursday,True,True
538719,581413,23535,WALL ART BICYCLE SAFETY,4,2011-12-08 14:39:00,5.95,16833,United Kingdom,False,2011,12,Thursday,True,True
540066,581457,23535,WALL ART BICYCLE SAFETY,126,2011-12-08 18:43:00,3.90,18102,United Kingdom,False,2011,12,Thursday,True,True


1C. Decsription
czy są nany?
jest 1451 nanow, nie uznaje tego za bład, niektore produkty moga nie miec opisu bo są np. dopiero dodawane

In [278]:
isNansDecsription = data['Description'].isna().sum()
isNansDecsription

np.int64(1454)

sprawdzam ilość unikalnych opisów
4223 unikalnych opisów na 4070 unikalnych kodów oznacza to jeden kod moze byc przypisany do np roznych kolorow danego produktu

In [279]:
nUniqueDescription = data['Description'].nunique()
nUniqueDescription

4223

1D. Quantity
Czy są nany?
nie ma nanów

In [280]:
isNansQuantity = data['Quantity'].isna().sum()
isNansQuantity

np.int64(0)

Czy są wartości ujemne?
tak są i dla zwyklych prodotkow, oznaczaja one po description ze są to np straty produktow typu popsute, i sa negatywne dla zwrotow discount

In [281]:
isNegativeQuantity = data[data['Quantity'] < 0]
isNegativeQuantity


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom,True
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom,True
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom,True
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom,True
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom,True
...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom,True
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom,True
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom,True
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom,True


In [282]:
isNegativeQuantityWithoutC = isNegativeQuantity[isNegativeQuantity['InvoiceNo'].str[0] != 'C']
isNegativeQuantityWithoutC

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom,False
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,NaN,United Kingdom,False
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom,False
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom,False
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,NaN,United Kingdom,False
...,...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,NaN,United Kingdom,False
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,NaN,United Kingdom,False
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,NaN,United Kingdom,False
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,NaN,United Kingdom,False


Czy jest quantity równe zero?
nie ma rekordow z quantity rowny -

In [283]:
isZeroQuantity = data[data['Quantity'] == 0]
isZeroQuantity

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund


Czy są ekstrymalnie dziwne wartosci
Są dwie 80995 oraz 74215, tylko ze sa one nastepnie na minusie musial to byc jakis blad zamowienia.

In [284]:
isHighQuantity = data['Quantity'].value_counts().sort_index(ascending=False)
isHighQuantity

Quantity
 80995    1
 74215    1
 12540    1
 5568     1
 4800     1
         ..
-9058     1
-9360     1
-9600     2
-74215    1
-80995    1
Name: count, Length: 722, dtype: int64

Jaka jest rozkład ilości?
wiekszosc to sa male ilosci

In [356]:
quantityValueCounts = data['Quantity'].value_counts(ascending=False)
quantityValueCounts

Quantity
 1        73314
 12       60033
 2        58003
 6        37688
 4        32183
          ...  
 912          1
 1404         1
 698          1
 80995        1
-80995        1
Name: count, Length: 436, dtype: int64

1E.InvoiceDate

Czy jest wlasciwy typ danych dla tej kolumny?
nie,zmieniam typ na datetime z obiekt

In [ ]:
data['InvoiceDate'] =  data['InvoiceDate'].astype('datetime64[s]')



/tmp/ipykernel_10988/3508754278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['InvoiceDate'] =  data['InvoiceDate'].astype('datetime64[s]')


InvoiceNo         string[python]
StockCode         string[python]
Description               object
Quantity                   int64
InvoiceDate        datetime64[s]
UnitPrice                float64
CustomerID                 Int32
Country                   object
isRefund                 boolean
InvoiceYear                int32
InvoiceMonth               int32
InvoiceWeekday            object
IsWeekend                   bool
IsUK                        bool
dtype: object

Czy są nany?
nie ma nanów

In [286]:
isNansInvoiceDate = data['InvoiceDate'].isna().sum()
isNansInvoiceDate

np.int64(0)

 – Jaki jest najstarszy i najnowszy InvoiceDate?
najstarszy to 2010-12-01,najnowyszy 2011-12-09
 – Czy dane obejmują cały rok, czy tylko wycinek?
dane obejumja caly rok i 8 dni

In [287]:
invoiceDateMax = data['InvoiceDate'].max()
invoiceDateMin = data['InvoiceDate'].min()
listWithMinMax = list()
listWithMinMax.append([invoiceDateMax,invoiceDateMin])
listWithMinMax

[[Timestamp('2011-12-09 12:50:00'), Timestamp('2010-12-01 08:26:00')]]

tworze zmienne pomocnicze czyli feature engineering

In [288]:
data['InvoiceYear'] = data['InvoiceDate'].dt.year
data['InvoiceMonth'] = data['InvoiceDate'].dt.month
data['InvoiceWeekday'] = data['InvoiceDate'].dt.day_name()
data['IsWeekend'] = data['InvoiceDate'].dt.day > 4
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund,InvoiceYear,InvoiceMonth,InvoiceWeekday,IsWeekend
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,False,2010,12,Wednesday,False
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,False,2010,12,Wednesday,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,False,2010,12,Wednesday,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,False,2010,12,Wednesday,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,False,2010,12,Wednesday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,False,2011,12,Friday,True
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,False,2011,12,Friday,True
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,False,2011,12,Friday,True
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,False,2011,12,Friday,True


 Czy są wartości ekstremalne (np. wszystko z jednego dnia)?
 nie ma

In [289]:
invoiceDateValueCounts = data['InvoiceDate'].value_counts().sort_values(ascending=False)
invoiceDateValueCounts

InvoiceDate
2011-10-31 14:41:00    1114
2011-12-08 09:28:00     749
2011-12-09 10:03:00     731
2011-12-05 17:24:00     721
2011-06-29 15:58:00     705
                       ... 
2011-07-24 14:50:00       1
2011-07-24 13:56:00       1
2011-11-10 15:07:00       1
2011-07-22 16:52:00       1
2011-07-24 13:14:00       1
Name: count, Length: 23260, dtype: int64

1F. Unit Price
czy jest prawidłowy typ danych?
tak typ danych tej kolumny to float

czy są nany?
0 nanów

In [290]:
isNansUnitPrice = data['UnitPrice'].isna().sum()
isNansUnitPrice

np.int64(0)

Czy są wartości ujemne?
są dwie spowodowane jakimś błedem

In [291]:
isNegativeUnitPriceValues = data[data['UnitPrice'] < 0]
isNegativeUnitPriceValues

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isRefund,InvoiceYear,InvoiceMonth,InvoiceWeekday,IsWeekend
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,False,2011,8,Friday,True
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,False,2011,8,Friday,True


Czy są wartości ekstremalne?
są 4 wartosci powyzej 16 tysiecy za jeden produkt, jest tez 2515 rekordow o unitprice 0.są to pewnie ceny po rabacie jakims.

In [292]:
unitPriceValueCounts = data['UnitPrice'].value_counts().sort_index(ascending=False)
unitPriceValueCounts

UnitPrice
 38970.000       1
 17836.460       1
 16888.020       1
 16453.710       1
 13541.330       3
              ... 
 0.030           3
 0.010           1
 0.001           4
 0.000        2515
-11062.060       2
Name: count, Length: 1630, dtype: int64

ile jest unikalnych wartości
jest 1630 unikalnych wartosci


In [293]:
nUniqueUnitPrice = data['UnitPrice'].nunique()
nUniqueUnitPrice

1630

1G. CustomerID

czy są nan?
Usuwam rekordy bez CustomerID (~25%), ponieważ są niemożliwe do przypisania do klienta.
Analiza poprzednich kolumn wykazała, że często zawierają błędy np.manualne wpisy

In [294]:
isNansCustomerID = data['CustomerID'].isna().sum()
isNansCustomerID
data = data[~data['CustomerID'].isna()]

Czy typ danych jest odpowiedni?
Nie, jest float, zmieniam na integer

In [295]:
data['CustomerID'] = data['CustomerID'].astype('Int32')
data.dtypes

/tmp/ipykernel_10988/3307269198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CustomerID'] = data['CustomerID'].astype('Int32')


InvoiceNo         string[python]
StockCode         string[python]
Description               object
Quantity                   int64
InvoiceDate        datetime64[s]
UnitPrice                float64
CustomerID                 Int32
Country                   object
isRefund                 boolean
InvoiceYear                int32
InvoiceMonth               int32
InvoiceWeekday            object
IsWeekend                   bool
dtype: object

Czy występuja nietypowe wartosci?
Czy dane są ciągłe? (czy ID rosną bez dziur?)
nie, poniewaz udalo sie zmienic kolumne na int czyli nie wystepuje nic oprocz liczb calkowitych.Czy są powtórzenia / duplikaty?
No są jakby moglo ich nie byc. przeciez jeden klient moze miec wiele zamowien

1H. Country

czy są nany?
Nie ma

In [296]:
isNansCountry = data['Country'].isna().sum()
isNansCountry

np.int64(0)

czy są podejrzanie wartosci?
nie ma
Czy występują różne warianty zapisu jednego kraju?
Niepotrzebna jest standaryzcja

In [297]:
valueCountsCountry = data['Country'].value_counts().sort_values(ascending=True).head(50)
valueCountsCountry

Country
Saudi Arabia                10
Bahrain                     17
Czech Republic              30
Brazil                      32
Lithuania                   35
Lebanon                     45
RSA                         58
European Community          61
United Arab Emirates        68
Malta                      127
Greece                     146
Canada                     151
Iceland                    182
Singapore                  229
Unspecified                244
Israel                     250
USA                        291
Poland                     341
Japan                      358
Denmark                    389
Austria                    401
Sweden                     462
Cyprus                     622
Finland                    695
Channel Islands            758
Italy                      803
Norway                    1086
Australia                 1259
Portugal                  1480
Switzerland               1877
Belgium                   2069
Netherlands               2371


Czy większość danych pochodzi z jednego kraju (np. UK)?
Tak wiekszosc z uk. stworze kolumne IsUK

In [298]:
data['IsUK'] = data['Country'].isin(['United Kingdom'])


/tmp/ipykernel_10988/2115454288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['IsUK'] = data['Country'].isin(['United Kingdom'])


na sam koniec dodaje kolumne totalPrice to przyszlej analizy

In [ ]:
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']
data